In [1]:
import nltk 
import numpy as np
import pandas as pd
import tensorflow as tf
import re 
import os 
import unicodedata
import zipfile
import data_load
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

### Load Data

In [2]:
data_load.clean_dir('data')

Cleaning directory data...


In [3]:
NUM_SENT_PAIRS = 30000

download_url = "http://www.manythings.org/anki/fra-eng.zip"
en_sents, fr_sents_in, fr_sents_out = data_load.download_and_read_url(download_url, num_sent_pairs=NUM_SENT_PAIRS)

### Attention layer

In [4]:
class AdditiveAttention(tf.keras.layers.Layer):
    def __init__(self, num_units):
        super(AdditiveAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(num_units)
        self.W2 = tf.keras.layers.Dense(num_units)
        self.V  = tf.keras.layers.Dense(1)
        
    def call(self, query, values):
        query_with_timesteps = tf.expand_dims(query, axis=1)
        score = self.V(
            tf.keras.activations.tanh(self.W1(query_with_timesteps) + self.W2(values))
        )
        alignment = tf.nn.softmax(score, axis=1)
        context = tf.reduce_sum(
            tf.linalg.matmul(
                tf.linalg.matrix_transpose(alignment),
                values
            ), axis=1
        )
        context = tf.expand_dims(context, axis=1)
        return context, alignment 
    
class Encoder(tf.keras.models.Model):
    def __init__(self, vocab_size, embedding_dim, num_timesteps, encoder_dim, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.encoder_dim = encoder_dim
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=num_timesteps)
        self.rnn = tf.keras.layers.GRU(encoder_dim, return_sequences=True, return_state=True)
        
    def call(self, x, state):
        x = self.embedding(x)
        x, state = self.rnn(x, initial_state=state)
        return x, state
    
    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.encoder_dim))
    
class Decoder(tf.keras.models.Model):
    def __init__(self, vocab_size, embedding_dim, num_timesteps, decoder_dim, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.decoder_dim = decoder_dim
        self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=num_timesteps)
        self.attention = AdditiveAttention(embedding_dim)
        self.rnn = tf.keras.layers.GRU(decoder_dim, return_sequences=True, return_state=True)
        self.Wc = tf.keras.layers.Dense(decoder_dim, activation='tanh')
        self.Ws = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x, state, encoder_out):
        x = self.embedding(x)
        context, alignment = self.attention(x, encoder_out)
        x = tf.expand_dims(
            tf.concat([
                x, tf.squeeze(context, axis=1)
            ], axis=1),
        axis=1)
        x, state = self.rnn(x, state)
        x = self.Wc(x)
        x = self.Ws(x)
        return x, state, alignment
    
def loss_fn(ytrue, ypred):
    scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    mask = tf.math.logical_not(tf.math.equal(ytrue, 0))
    mask = tf.cast(mask, dtype=tf.int64)
    loss = scce(ytrue, ypred, sample_weight=mask)
    return loss

@tf.function
def train_step(encoder_in, decoder_in, decoder_out, encoder_state):
    with tf.GradientTape() as tape:
        encoder_out, encoder_state = encoder(encoder_in, encoder_state)
        decoder_state = encoder_state
        
        loss = 0
        for t in range(decoder_out.shape[1]):
            decoder_in_t = decoder_in[:, t]
            decoder_pred_t, decoder_state, _ = decoder(decoder_in_t, decoder_state, encoder_out)
            loss += loss_fn(decoder_out[:, t], decoder_pred_t)
    
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss / decoder_out.shape[1]

### Tokenization and data set up 

In [5]:
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters="", lower=False)
en_tokenizer.fit_on_texts(en_sents)
en_data = en_tokenizer.texts_to_sequences(en_sents)
en_data = tf.keras.preprocessing.sequence.pad_sequences(en_data, padding='post')

fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters="", lower=False)
fr_tokenizer.fit_on_texts(fr_sents_in)
fr_tokenizer.fit_on_texts(fr_sents_out)
fr_data_in = fr_tokenizer.texts_to_sequences(fr_sents_in)
fr_data_in = tf.keras.preprocessing.sequence.pad_sequences(fr_data_in, padding='post')
fr_data_out = fr_tokenizer.texts_to_sequences(fr_sents_out)
fr_data_out = tf.keras.preprocessing.sequence.pad_sequences(fr_data_out, padding='post')

en_maxlen = en_data.shape[1]
fr_maxlen = fr_data_out.shape[1]
print(f"seqlen (en): {en_maxlen}, seqlen (fr): {fr_maxlen}")

seqlen (en): 8, seqlen (fr): 16


In [6]:
EMBEDDING_DIM = 32
ENCODER_DIM, DECODER_DIM = 64, 64
BATCH_SIZE = 16
NUM_EPOCHS = 30

In [7]:
tf.random.set_seed(123)

batch_size = BATCH_SIZE
dataset = tf.data.Dataset.from_tensor_slices((en_data, fr_data_in, fr_data_out))
dataset = dataset.shuffle(10000)
test_size = NUM_SENT_PAIRS // 4
test_dataset = dataset.take(test_size).batch(batch_size, drop_remainder=True)
train_dataset = dataset.skip(test_size).batch(batch_size, drop_remainder=True)

en_vocab_size = len(en_tokenizer.word_index)
fr_vocab_size = len(fr_tokenizer.word_index)
en_word2idx = en_tokenizer.word_index
en_idx2word = {idx:word for word, idx in en_word2idx.items()}
fr_word2idx = fr_tokenizer.word_index
fr_idx2word = {idx:word for word, idx in fr_word2idx.items()}
print(f"vocab size (en): {en_vocab_size}, vocab size (fr): {fr_vocab_size}")

vocab size (en): 4354, vocab size (fr): 7635


In [8]:
encoder = Encoder(en_vocab_size+1, embedding_dim=EMBEDDING_DIM, num_timesteps=en_maxlen, encoder_dim=ENCODER_DIM)
decoder = Decoder(fr_vocab_size+1, embedding_dim=EMBEDDING_DIM, num_timesteps=fr_maxlen, decoder_dim=DECODER_DIM)

### Test code to examine dimensions

In [9]:
for encoder_in, decoder_in, decoder_out in train_dataset:
    print("inputs:", encoder_in.shape, decoder_in.shape, decoder_out.shape)
    encoder_state = encoder.init_state(batch_size)
    encoder_out, encoder_state = encoder(encoder_in, encoder_state)
    decoder_state = encoder_state
    decoder_pred = []
    for t in range(decoder_out.shape[1]):
        decoder_in_t = decoder_in[:, t]
        decoder_pred_t, decoder_state, _ = decoder(decoder_in_t,
            decoder_state, encoder_out)
        decoder_pred.append(decoder_pred_t.numpy())
    decoder_pred = tf.squeeze(np.array(decoder_pred), axis=2)
    break
print("encoder input          :", encoder_in.shape)
print("encoder output         :", encoder_out.shape, "state:", encoder_state.shape)
print("decoder output (logits):", decoder_pred.shape, "state:", decoder_state.shape)
print("decoder output (labels):", decoder_out.shape)

inputs: (16, 8) (16, 16) (16, 16)
encoder input          : (16, 8)
encoder output         : (16, 8, 64) state: (16, 64)
decoder output (logits): (16, 16, 7636) state: (16, 64)
decoder output (labels): (16, 16)


### Training and Inference

In [10]:
def predict(encoder, decoder, batch_size, en_sents, en_data, fr_sents_out, fr_word2idx, fr_idx2word):
    random_id = np.random.choice(len(en_sents))
    print("Input      :    ", " ".join(en_sents[random_id]))
    print("Label      :    ", " ".join(fr_sents_out[random_id]))
    
    
    encoder_in = tf.expand_dims(en_data[random_id], axis=0)
    encoder_state = encoder.init_state(1)
    encoder_out, encoder_state = encoder(encoder_in, encoder_state)
    decoder_state = encoder_state
    
    fr_pred_sent = []
    decoder_in = tf.expand_dims(tf.constant(fr_word2idx["BOS"]), axis=0)
    
    while True:
        decoder_pred, decoder_state, alignment = decoder(decoder_in, decoder_state, encoder_out)
        decoder_pred = tf.argmax(decoder_pred, axis=-1)
        pred_word = fr_idx2word[decoder_pred.numpy()[0][0]]
        fr_pred_sent.append(pred_word)
        if pred_word == "EOS":
            break
        decoder_in = tf.squeeze(decoder_pred, axis=1)
        
    print("Predicted  :    ", " ".join(fr_pred_sent))
    
def evaluate_bleu_score(encoder, decoder, test_dataset, fr_word2idx, fr_idx2word):
    bleu_scores = []
    smoothing_fn = SmoothingFunction()
    
    for encoder_in, decoder_in, decoder_out in test_dataset:
        encoder_state = encoder.init_state(batch_size)
        encoder_out, encoder_state = encoder(encoder_in, encoder_state)
        decoder_state = encoder_state
        
        ref_sent_ids = np.zeros_like(decoder_out)
        hyp_sent_ids = np.zeros_like(decoder_out)
        for t in range(decoder_out.shape[1]):
            decoder_out_t = decoder_out[:, t]
            decoder_in_t = decoder_in[:, t]
            decoder_pred_t, decoder_state, alignment = decoder(decoder_in_t, decoder_state, encoder_out)
            decoder_pred_t = tf.argmax(decoder_pred_t, axis=-1)
            for b in range(decoder_out.shape[0]):
                ref_sent_ids[b, t] = decoder_out_t.numpy()[0]
                hyp_sent_ids[b, t] = decoder_pred_t.numpy()[0][0]
        
        for b in range(ref_sent_ids.shape[0]):
            ref_sent = [fr_idx2word[i] for i in ref_sent_ids[b] if i > 0] 
            hyp_sent = [fr_idx2word[i] for i in ref_sent_ids[b] if i > 0]
            ref_sent = ref_sent[0:-1]
            hyp_sent = hyp_sent[0:-1]
            
            bleu_score = sentence_bleu([ref_sent], hyp_sent, smoothing_function=smoothing_fn.method1)
            bleu_scores.append(bleu_score)
    
    return np.mean(np.array(bleu_scores))

In [11]:
optimizer = tf.keras.optimizers.Adam()
num_epochs = NUM_EPOCHS

for e in range(num_epochs):
    encoder_state = encoder.init_state(batch_size)
    
    for batch, data in enumerate(train_dataset):
        encoder_in, decoder_in, decoder_out = data
        loss = train_step(encoder_in, decoder_in, decoder_out, encoder_state)
    
    print("="*30)
    print("EPOCH: {}, LOSS: {:.4f}".format(e+1, loss.numpy()))
    print("-"*30)
    
    predict(encoder, decoder, batch_size, en_sents, en_data, fr_sents_out, fr_word2idx, fr_idx2word)
    eval_score = evaluate_bleu_score(encoder, decoder, test_dataset, fr_word2idx, fr_idx2word)
    print("EVAL (BLEU):    ", "{:.3e}".format(eval_score))
    print("-"*30)

EPOCH: 1, LOSS: 1.5176
------------------------------
Input      :     he slept all day .
Label      :     il a dormi toute la journee . EOS
Predicted  :     tom est un air . EOS
EVAL (BLEU):     9.316e-01
------------------------------
EPOCH: 2, LOSS: 1.2458
------------------------------
Input      :     we re resilient .
Label      :     nous sommes endurantes . EOS
Predicted  :     nous sommes tres . EOS
EVAL (BLEU):     8.967e-01
------------------------------
EPOCH: 3, LOSS: 1.0178
------------------------------
Input      :     get away !
Label      :     cassez vous . EOS
Predicted  :     veuillez ! EOS
EVAL (BLEU):     9.320e-01
------------------------------
EPOCH: 4, LOSS: 0.9991
------------------------------
Input      :     fix the clock .
Label      :     reparez l horloge . EOS
Predicted  :     laisse le monde ! EOS
EVAL (BLEU):     9.072e-01
------------------------------
EPOCH: 5, LOSS: 0.9447
------------------------------
Input      :     nice to meet you .
Label   